In [1]:
import pandas as pd
import numpy as np

# Corregir la ruta del archivo
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas208semanas.parquet")
df_p = pd.read_parquet("/home/donsson/proyectos/API/costo_productos.parquet")


# COMPLEMENTAR INFO

In [2]:
import re
import unicodedata

# Diccionario de códigos a sucursales
mapa_codigos = {
    "FCAL": "CALI",
    "FMDE": "MEDELLIN",
    "FBOG": "BOGOTA",
    "FCTG": "CARTAGENA",
    "FBAQ": "BARRANQUILLA",
    "FVAL":"VALLADOLID"
}


# Equivalencias para normalizar nombres truncados o mal escritos
mapa_equivalencias = {
    "MEDELLIN": "MEDELLIN",
    "MEDELLI": "MEDELLIN",
    "MEDELL": "MEDELLIN",
    "MEDELI": "MEDELLIN",
    "CALI": "CALI",
    "BARRANQUILLA": "BARRANQUILLA",
    "BOGOTA": "BOGOTA",
    "CARTAGENA": "CARTAGENA",
    "VALLADOLID": "VALLADOLID"
}

def normalizar(texto):
    """Quita tildes y pasa a mayúsculas"""
    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join([c for c in texto if not unicodedata.combining(c)])
    return texto.upper()

def extraer_sucursal(nombre):
    if not isinstance(nombre, str):
        return "VENDEDOR EXTERNO"
    
    sucursal = None
    
    # 1) Buscar "Mostrador ..."
    match = re.search(r"Mostrador\s+([A-Za-z0-9\s]+)", nombre, re.IGNORECASE)
    if match:
        sucursal = match.group(1).strip()
    else:
        # 2) Buscar "Calle" o "Cota"
        match2 = re.search(r"(Calle\s+\d+|Cota)", nombre, re.IGNORECASE)
        if match2:
            sucursal = match2.group(1).strip()
        else:
            # 3) Buscar prefijo de código
            for prefijo, ciudad in mapa_codigos.items():
                if nombre.upper().startswith(prefijo):
                    return ciudad
            return "VENDEDOR EXTERNO"
    
    # Normalizar texto
    sucursal = normalizar(sucursal)
    
    # Limpiar T1, T2, T3 al final
    sucursal = re.sub(r"\s*T\d+$", "", sucursal).strip()
    
    # Aplicar equivalencias
    sucursal = mapa_equivalencias.get(sucursal, sucursal)
    
    return sucursal

# Aplicar al dataframe
df["Sucursal"] = df["invoice_name"].apply(extraer_sucursal)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654763 entries, 0 to 654762
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   price_subtotal  654763 non-null  float64       
 1   company_id      654763 non-null  object        
 2   id              654763 non-null  int64         
 3   quantity        654763 non-null  float64       
 4   product_id_num  654763 non-null  int64         
 5   product_name    654763 non-null  object        
 6   invoice_id_num  654763 non-null  int64         
 7   invoice_name    654763 non-null  object        
 8   date_invoice    654763 non-null  datetime64[ns]
 9   Sucursal        654763 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 50.0+ MB


In [4]:
df["date_invoice"].min()
df["date_invoice"].max()

Timestamp('2025-09-10 13:11:40')

## UNIR COSTO DEL PRODUCTO

In [5]:
df_unido = pd.merge(df,df_p,on='product_name',how="left")

In [6]:
df_unido = df_unido[df_unido["Sucursal"]!= "VENDEDOR EXTERNO"]
df_def = df_unido.drop_duplicates(
    subset=["product_name", "quantity", "invoice_id_num", "date_invoice"],
    keep="first"   # conserva la primera aparición
).reset_index(drop=True)

df_def = df_def[df_def["producto_costo_unitario"]!=0 ] #Eliminar porductos con costo en 0


df_def = df_def[["quantity","product_name","date_invoice","Sucursal","producto_costo_unitario"]]

# Aseguramos que la columna sea datetime (por si acaso)
df_def["date_invoice"] = pd.to_datetime(df_def["date_invoice"])


## BUSCADOR

In [7]:
df_def[(df_def["Sucursal"]=="MEDELLIN")&( df_def["product_name"].str.contains("BLS00037125"))]

,quantity,product_name,date_invoice,Sucursal,producto_costo_unitario
3,2.0,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",2025-09-10 13:10:28,MEDELLIN,32797.97
203,1.0,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",2025-09-09 20:18:54,MEDELLIN,32797.97
673,1.0,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",2025-09-09 13:39:47,MEDELLIN,32797.97
1623,2.0,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",2025-09-05 18:44:54,MEDELLIN,32797.97
2004,1.0,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",2025-09-05 14:04:10,MEDELLIN,32797.97
...,...,...,...,...,...
441454,1.0,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",2021-09-22 16:29:21,MEDELLIN,32797.97
442272,2.0,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",2021-09-20 17:42:20,MEDELLIN,32797.97
442481,1.0,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",2021-09-18 16:07:05,MEDELLIN,32797.97
442869,1.0,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",2021-09-17 14:51:53,MEDELLIN,32797.97


In [8]:
prueba = df_def[df_def["Sucursal"]=="BARRANQUILLA"]
#prueba.to_excel("mirar.xlsx")
ventas_semana = (
    prueba
    .groupby(pd.Grouper(key="date_invoice", freq="W"))["quantity"]
    .sum()
    .reset_index()
)

ventas_mes = (
    prueba
    .groupby(pd.Grouper(key="date_invoice", freq="ME"))["quantity"]
    .sum()
    .reset_index()
)


## EMA MENSUAL

In [9]:
import pandas as pd

# --- 1. Crear columna de Mes ---
prueba["Mes"] = prueba["date_invoice"].dt.to_period("M").dt.to_timestamp()

# --- 2. Agrupar por Sucursal + Producto + Mes ---
df_mensual = (prueba.groupby(["Sucursal", "product_name", "Mes"], as_index=False)
                     .agg({
                         "quantity": "sum",
                         "producto_costo_unitario": "last"
                     }))

def calcular_ema(grupo, alpha=0.2):
    grupo = grupo.sort_values("Mes").reset_index(drop=True)

    # EMA inicial = promedio de los primeros 4 meses
    if len(grupo) >= 4:
        ema0 = grupo.loc[:3, "quantity"].mean()
    else:
        ema0 = grupo["quantity"].mean()  # fallback si hay menos de 4 meses

    ema_vals = [ema0]

    # EMA desde el 5to mes en adelante
    for t in range(1, len(grupo)):
        xt = grupo.loc[t, "quantity"]
        ema_prev = ema_vals[-1]
        ema_t = alpha * xt + (1 - alpha) * ema_prev
        ema_vals.append(ema_t)

    grupo["EMA"] = ema_vals
    return grupo


# --- Aplicar por Sucursal + Producto ---
df_mensual = (
    df_mensual
    .groupby(["Sucursal", "product_name"], group_keys=False)
    .apply(calcular_ema)
    .reset_index(drop=True)  # limpiar índices
)


# --- 5. Ajuste de nombres de columnas como en tu sistema ---
df_final = df_mensual.rename(columns={
    "Sucursal": "Sucursal",
    "product_name": "Producto",
    "producto_costo_unitario": "Costo unitario"
})

# Aquí deberías enlazar también tus catálogos de productos
# para agregar "Descripción" y "Código producto"
# (ej. con un merge si tienes otro DataFrame de catálogo)

df_final = df_final[["Mes","Sucursal","Producto","Costo unitario","quantity","EMA"]]


df_final.sort_values(by="Mes")

/tmp/ipykernel_114186/1070849411.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba["Mes"] = prueba["date_invoice"].dt.to_period("M").dt.to_timestamp()
/tmp/ipykernel_114186/1070849411.py:39: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calcular_ema)


,Mes,Sucursal,Producto,Costo unitario,quantity,EMA
15412,2021-09-01,BARRANQUILLA,[BLS10077125] GS077A FILTRO ACEITE BY PASS VOL...,58669.67,3.0,4.750000
6854,2021-09-01,BARRANQUILLA,[BCS00638125] GS638 FILTRO SEGUNDARIO CON DREN...,46717.59,1.0,6.250000
10139,2021-09-01,BARRANQUILLA,"[BHS00435125] GS435 FILTRO HIDRAULICO FORD, NE...",37772.59,6.0,26.250000
13392,2021-09-01,BARRANQUILLA,[BLS00232125] GS232 FILTRO ACEITE BALDWIN - NI...,14106.35,35.0,61.250000
952,2021-09-01,BARRANQUILLA,"[BAP09109125] DA9109 FILTRO AIRE 2º JCB , NEW ...",79779.67,1.0,5.500000
...,...,...,...,...,...,...
17629,2025-09-01,BARRANQUILLA,[DAB02926025] DA2926 FILTRO AIRE DONSSON - INT...,38199.00,1.0,23.514386
4214,2025-09-01,BARRANQUILLA,[BCS00236125] GS236 FILTRO SEPARADOR COMBUSTIB...,37345.78,3.0,24.416652
17700,2025-09-01,BARRANQUILLA,[DAB02933025] DA2933 FILTRO AIRE NISSAN D22 FR...,15214.08,1.0,2.972145
17790,2025-09-01,BARRANQUILLA,"[DAB02946025] DA2946 FILTRO AIRE JCB,TEREX,M.F...",17678.37,16.0,18.775588


## EMA SEMANAL 

In [10]:


# --- 1. Crear columna de Semana ---
prueba["Semana"] = prueba["date_invoice"].dt.to_period("W").dt.start_time

# --- 2. Filtrar solo las últimas X semanas ---
max_semana = prueba["Semana"].max()
min_semana = max_semana - pd.Timedelta(weeks=11)  # 16 semanas contando la actual
prueba_16 = prueba[prueba["Semana"] >= min_semana]

# --- 3. Agrupar por Sucursal + Producto + Semana ---
df_semanal = (
    prueba_16.groupby(["Sucursal", "product_name", "Semana"], as_index=False)
             .agg({
                 "quantity": "sum",
                 "producto_costo_unitario": "last"
             })
)

# --- 4. Función para calcular EMA semanal ---
def calcular_ema_semanal(grupo, alpha=0.2):
    grupo = grupo.sort_values("Semana").reset_index(drop=True)

    # EMA inicial: promedio de primeras 4 semanas (o menos si no hay tantas)
    ema0 = grupo.loc[:3, "quantity"].mean() if len(grupo) >= 4 else grupo["quantity"].mean()
    
    ema_vals = [ema0]
    for t in range(1, len(grupo)):
        xt = grupo.loc[t, "quantity"]
        ema_prev = ema_vals[-1]
        ema_t = alpha * xt + (1 - alpha) * ema_prev
        ema_vals.append(ema_t)
    
    grupo["EMA"] = ema_vals
    return grupo

# --- 5. Aplicar EMA por producto y sucursal ---
df_semanal = (
    df_semanal.groupby(["Sucursal", "product_name"], group_keys=False)
              .apply(calcular_ema_semanal)
)

# --- 6. Extra: Año y número de semana (para que quede como Odoo) ---
df_semanal["Año"] = df_semanal["Semana"].dt.year
df_semanal["NroSemana"] = df_semanal["Semana"].dt.isocalendar().week


/tmp/ipykernel_114186/3803341931.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba["Semana"] = prueba["date_invoice"].dt.to_period("W").dt.start_time
/tmp/ipykernel_114186/3803341931.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calcular_ema_semanal)


In [11]:
df_semanal

,Sucursal,product_name,Semana,quantity,producto_costo_unitario,EMA,Año,NroSemana
0,BARRANQUILLA,[AHBNTO68030] ACEITE MOBIL HIDRAULICO NUTO 68 ...,2025-07-28,1.0,342304.08,1.000000,2025,31
0,BARRANQUILLA,[ALB15W40131] ACEITE MOBIL MX15W40 DELVAC x GA...,2025-06-23,8.0,339158.35,9.250000,2025,26
1,BARRANQUILLA,[ALB15W40131] ACEITE MOBIL MX15W40 DELVAC x GA...,2025-06-30,4.0,339158.35,8.200000,2025,27
2,BARRANQUILLA,[ALB15W40131] ACEITE MOBIL MX15W40 DELVAC x GA...,2025-07-07,16.0,339158.35,9.760000,2025,28
3,BARRANQUILLA,[ALB15W40131] ACEITE MOBIL MX15W40 DELVAC x GA...,2025-07-14,9.0,339158.35,9.608000,2025,29
...,...,...,...,...,...,...,...,...
3,BARRANQUILLA,[DRGCOOLV035] REFRIGERANTE HIDROCOOL ANTICORRO...,2025-07-21,10.0,13412.98,5.008000,2025,30
4,BARRANQUILLA,[DRGCOOLV035] REFRIGERANTE HIDROCOOL ANTICORRO...,2025-07-28,1.0,13412.98,4.206400,2025,31
5,BARRANQUILLA,[DRGCOOLV035] REFRIGERANTE HIDROCOOL ANTICORRO...,2025-08-04,2.0,13412.98,3.765120,2025,32
6,BARRANQUILLA,[DRGCOOLV035] REFRIGERANTE HIDROCOOL ANTICORRO...,2025-08-25,1.0,13412.98,3.212096,2025,35


In [14]:
import pandas as pd

# --- 1. Crear columna de Semana ---
prueba["Semana"] = prueba["date_invoice"].dt.to_period("W").dt.start_time
prueba["Año"] = prueba["date_invoice"].dt.isocalendar().year
prueba["NroSemana"] = prueba["date_invoice"].dt.isocalendar().week

# --- 2. Parámetro: semana y año de referencia ---
semana_ref = 35
anio_ref = 2025
num_semanas = 16

# --- 3. Determinar rango de semanas ---
fecha_ref = pd.to_datetime(f"{anio_ref}-W{semana_ref}-1", format="%G-W%V-%u")
fecha_inicio = fecha_ref - pd.Timedelta(weeks=num_semanas-1)

# --- 4. Filtrar datos ---
prueba_filtro = prueba[(prueba["Semana"] >= fecha_inicio) & (prueba["Semana"] <= fecha_ref)]

# --- 5. Agrupar por Sucursal + Producto + Semana ---
df_semanal = (
    prueba_filtro.groupby(["Sucursal", "product_name", "Semana"], as_index=False)
                 .agg({
                     "quantity": "sum",
                     "producto_costo_unitario": "last"
                 })
)

# --- 6. Función para calcular EMA con EMA0 = promedio primeras 4 semanas ---
def calcular_ema(grupo, alpha=0.2):
    grupo = grupo.sort_values("Semana").reset_index(drop=True)
    # EMA inicial: promedio primeras 4 semanas (si hay menos, promedio todas)
    ema0 = grupo.loc[:3, "quantity"].mean() if len(grupo) >= 4 else grupo["quantity"].mean()
    ema_vals = [ema0]
    for t in range(1, len(grupo)):
        xt = grupo.loc[t, "quantity"]
        ema_vals.append(alpha * xt + (1 - alpha) * ema_vals[-1])
    grupo["EMA"] = ema_vals
    return grupo

# --- 7. Aplicar EMA ---
df_semanal = (
    df_semanal.groupby(["Sucursal", "product_name"], group_keys=False)
              .apply(calcular_ema)
)

# --- 8. Añadir año y número de semana ISO ---
df_semanal["Año"] = df_semanal["Semana"].dt.isocalendar().year
df_semanal["NroSemana"] = df_semanal["Semana"].dt.isocalendar().week

# --- 9. Filtrar para ver el resultado de la semana de referencia ---
df_final = df_semanal[df_semanal["NroSemana"] == 35]



/tmp/ipykernel_114186/3845857690.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba["Semana"] = prueba["date_invoice"].dt.to_period("W").dt.start_time
/tmp/ipykernel_114186/3845857690.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba["Año"] = prueba["date_invoice"].dt.isocalendar().year
/tmp/ipykernel_114186/3845857690.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [15]:
df_final = df_final.sort_values(by="EMA",ascending=False)
df_final.head(10)

,Sucursal,product_name,Semana,quantity,producto_costo_unitario,EMA,Año,NroSemana
15,BARRANQUILLA,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2025-08-25,44.0,13507.71,55.160890,2025,35
15,BARRANQUILLA,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",2025-08-25,41.0,32797.97,38.014752,2025,35
15,BARRANQUILLA,"[DAB02772025] DA2772 FILTRO AIRE BOBCAT, HITAC...",2025-08-25,9.0,10680.97,37.298852,2025,35
6,BARRANQUILLA,[DAB14570025] DA4570A FILTRO AIRE 2_ DONSSON -...,2025-08-25,17.0,9774.98,34.618432,2025,35
15,BARRANQUILLA,"[BCS00035125] GS035 FILTRO COMBUSTIBLE VOLVO,D...",2025-08-25,41.0,20638.72,28.048235,2025,35
10,BARRANQUILLA,[DAB14772025] DA4772A FILTRO AIRE INTERNO BOBC...,2025-08-25,7.0,9112.40,24.012054,2025,35
15,BARRANQUILLA,[BCS00025125] GS025 FILTRO COMBUSTIBLE CUMMINS...,2025-08-25,20.0,31058.67,23.038011,2025,35
4,BARRANQUILLA,[DAC10090189] DAC090A FILTRO AIRE CABINA TOYOT...,2025-08-25,24.0,3596.02,22.438400,2025,35
15,BARRANQUILLA,[DAB02666025] DA2666 FILTRO AIRE 1_ DONSSON -B...,2025-08-25,7.0,11371.65,21.461140,2025,35
7,BARRANQUILLA,[DCE00937189] G937 FILTRO COMBUSTIBLE TOYOTA H...,2025-08-25,30.0,6591.30,20.695066,2025,35
